In [1]:
import os, sys
os.environ['HADOOP_CONF_DIR'] = '/etc/hadoop/conf'
os.environ['YARN_CONF_DIR'] = '/etc/hadoop/conf'
os.environ['PYSPARK_PYTHON'] =  'python3.9'
os.environ['PYSPARK_DRIVER_PYTHON'] = 'python3.9'
os.environ['HADOOP_USER_NAME']='ssenigov'

from pyspark.sql import SparkSession
from pyspark import SparkContext, SparkConf

In [2]:
conf = SparkConf().setAppName('Posexplode').setMaster('yarn')\
   .set('spark.sql.legacy.timeParserPolicy', 'LEGACY')
spark = SparkSession.builder.config(conf=conf).getOrCreate()

sc = spark.sparkContext
print("app_id".ljust(40), sc.applicationId)

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/10/24 17:10:52 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/10/24 17:10:53 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
24/10/24 17:10:54 WARN DomainSocketFactory: The short-circuit local reads feature cannot be used because libhadoop cannot be loaded.
24/10/24 17:10:54 WARN Client: Neither spark.yarn.jars nor spark.yarn.archive is set, falling back to uploading libraries under SPARK_HOME.


app_id                                   application_1727681258360_0020


In [ ]:
from pyspark.sql.functions import col, concat, date_format, to_date, lit

df_list = spark.sql(" select array_repeat('dummy', {num_days}) as arr".format(num_days=31))\
    .selectExpr("posexplode(arr)").drop('col')\
    .withColumn('day', col('pos') + 1).drop('pos')\
    .withColumn('event_date', to_date(concat(lit('2024'), lit('10'), col('day')), 'yyyyMMdd'))\
    .withColumn('week_day', date_format(col('event_date'), 'EEEE').alias('week_day'))

df_list.show(n=31)

In [ ]:
spark.stop()
# df_list = spark.sql(" select repeat(',', {num_days}) as seq".format(num_days=31))\
    # .withColumn('arr', split(col('seq'), ','))\